In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from underthesea import word_tokenize, text_normalize
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding, Dropout

import re
import string
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
from sklearn import utils
tqdm.pandas(desc="progress-bar")
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, concatenate, Activation
from tensorflow.keras.models import Model
import random

In [23]:
data = pd.read_csv("data/labeled_data.csv")
data

,Unnamed: 0,Date,Title,open,high,low,close,volume_match
0,3,2022-01-04,"Bài toán hóc búa của Bitcoin, Chủ tịch nước: ""...",40.645833,41.158333,40.279167,40.741667,9.763350e+06
1,4,2022-01-05,BIDV giảm 50% phí giao dịch tài trợ thương mại...,40.754167,41.158333,40.104167,40.333333,8.901967e+06
2,5,2022-01-06,SSI: Giá cổ phiếu ngân hàng hiện chưa phản ánh...,40.029167,40.900000,39.575000,40.237500,1.082330e+07
3,6,2022-01-07,CLIP: Táo tợn dùng vật giống súng cướp chi nhá...,40.279167,40.616667,39.529167,39.904167,8.243367e+06
4,9,2022-01-10,"Bitcoin lao dốc mạnh xuống dưới 40.000 USD, đư...",39.850000,40.320833,39.029167,39.333333,7.876750e+06
...,...,...,...,...,...,...,...,...
642,944,2024-08-02,Công an triệt phá băng nhóm lừa đảo qua mạng q...,29.712500,30.229167,29.337500,30.070833,1.079972e+07
643,947,2024-08-05,Thủ tướng: NHNN thu hồi chỉ tiêu tăng trưởng t...,29.641667,29.808333,28.712500,28.958333,1.516620e+07
644,948,2024-08-06,SCB vừa tiếp tục đóng cửa nhiều phòng giao dịc...,29.200000,29.604167,28.862500,29.379167,8.852800e+06
645,949,2024-08-07,Ngân hàng nào đang trả lãi suất tiết kiệm trên...,29.450000,29.554167,28.975000,29.208333,7.648283e+06


In [24]:
emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"
                u"\U0001F300-\U0001F5FF" 
                u"\U0001F680-\U0001F6FF"  
                u"\U0001F1E0-\U0001F1FF"  
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE) # Unicode emojis.
vietnamese_stopwords = None
with open("data/vietnamese-stopwords.txt", "r", encoding="utf-8") as file:
    vietnamese_stopwords = file.read().splitlines()
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))
def clean_text(text):
    text = text.lower() # lowercase text
    
    text = re.sub(emoji_pattern, " ", text) # remove emojis
    
    text = re.sub(r'([a-z]+?)\1+',r'\1', text) # reduce repeated character (e.g. 'aaabbb' -> 'ab')
    
    # Ensure space before and after any punctuation mark
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])\s*(\w)", r"\1 \2 \3", text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])", r"\1 \2", text)
    
    text = re.sub(f"([{string.punctuation}])([{string.punctuation}])+",r"\1", text) # reduce consecutive punctuation
    
    # Remove any leading or trailing spaces, or leading or trailing punctuation marks from the text
    text = text.strip()
    while text.endswith(tuple(string.punctuation+string.whitespace)):
        text = text[:-1]
    while text.startswith(tuple(string.punctuation+string.whitespace)):
        text = text[1:]
        
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuation
        
    text = re.sub(r"\s+", " ", text) # reduce multiple spaces
    
    text = text_normalize(text) # make sure punctunation is in the right letter (Vietnamese case)
    text = word_tokenize(text, format="text") # tokenize the cleaned text
    # text = unidecode(text) # remove accent marks from sentences (no significant difference when accent marks is removed or kept)
    # Loại bỏ ký tự đặc biệt và số 
    processed_words = [re.sub(r'[^\w\s]', ' ', word).strip() for word in text if not re.search(r'\d', word)]
    # Loại bỏ '\t', '\n', ...
    processed_words = [re.sub(r'\s+', ' ', word).strip() for word in text if word]
    text = text.split(' ')
    tokens = [ word.replace('_', ' ') for word in text if word.lower().replace('_', ' ') not in vietnamese_stopwords]
    '''
    Proper tokenization allows models or algorithms to understand the semantics of the text better. 
    For instance, `"đi chơi" (go play)` should be treated as one unit 
    instead of two separate words "đi" (go) and "chơi" (play) for proper semantic understanding.
    '''
    return tokens

In [25]:
processed_docs = [clean_text(remove_punctuation(title)) for title in list(data["Title"])]
X = pd.Series(processed_docs)

In [26]:
def labelize_text_ug(title,label):
    result = []
    prefix = label
    for i, t in zip(title.index, title):
        result.append(TaggedDocument(t, [prefix + '_%s' % i]))
    return result

all_x_w2v = labelize_text_ug(X, 'all')

In [27]:
VECTOR_SIZE = 100

model = Word2Vec(sg=1, vector_size=VECTOR_SIZE, negative=5, window=2, min_count=2, workers=4, alpha=0.065, min_alpha=0.065)
model.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 647/647 [00:00<00:00, 3015238.54it/s]


In [28]:
# %%time
for epoch in range(60):
    model.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model.alpha -= 0.002

100%|██████████| 647/647 [00:00<00:00, 4162139.09it/s]


In [29]:
def get_document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)
X = pd.DataFrame(np.array([get_document_vector(doc, model) for doc in processed_docs]))
print(X.shape)

(647, 100)


In [30]:
final = pd.concat((data[["Date", "open", "high", "low", "close", "volume_match"]], X), axis=1)
final.to_csv("../model/data/vectorized_data.csv", index=False)